# 11 可视化
## 11.1 使用Visdom在 PyTorch 中进行可视化
Visdom是Facebook在2017年发布的一款针对PyTorch的可视化工具。官网,visdom由于其功能简单，一般会被定义为服务器端的matplot，也就是说我们可以直接使用python的控制台模式进行开发并在服务器上执行，将一些可视化的数据传送到Visdom服务上，通过Visdom服务进行可视化
### 安装
Visdom的安装很简单，直接使用命令
```pip install visdom```
安装即可。 在安装完成后，使用命令在本地启动服务器，启动后会提示It's Alive! You can navigate to http://localhost:8097 这就说明服务已经可用，我们打开浏览器，输入http://localhost:8097 即可看到页面。
```python -m visdom.server ```

端口8097是默认的端口可以在启动命令后加 -port参数指定端口，常用的参数还有 --hostname，-base_url等

### 基本概念
#### Environments
Environments的作用是对可视化区域进行分区，每个用户都会有一个叫做main的默认分区，在程序指定的情况下，默认的图表都会放到这里面
#### Panes
Panes是作为每一个可视化图表的容器，可以使用生成的图表，图片，文本进行填充，我们可以对Panes进行拖放，删除，调整大小和销毁等操作：  Panes和Environments是一对多的关系，即一个Environments可以包含多个Panes
#### VIEW
在对Panes进行调整后，可以通过VIEW对状态进行管理： 

#### 可视化接口
Visdom是由Plotly 提供的可视化支持，所以提供一下可视化的接口: - vis.scatter : 2D 或 3D 散点图 - vis.line : 线图 - vis.stem : 茎叶图 - vis.heatmap : 热力图 - vis.bar : 条形图 - vis.histogram: 直方图 - vis.boxplot : 箱型图 - vis.surf : 表面图 - vis.contour : 轮廓图 - vis.quiver : 绘出二维矢量场 - vis.image : 图片 - vis.text : 文本 - vis.mesh : 网格图 - vis.save : 序列化状态

#### 使用
绘制简单的图形
这里我们使用官方的DEMO来做样例

In [1]:
import torch
import math
import numpy as np
from visdom import Visdom
import time

In [2]:

env = Visdom() 
assert env.check_connection() #测试一下链接，链接错误的话会报错

Setting up a new session...


这里生成sin和cos两条曲线数据

In [3]:
Y = np.linspace(0, 2 * math.pi, 70)
X = np.column_stack((np.sin(Y), np.cos(Y)))

使用茎叶图展示

In [4]:
env.stem(
        X=X,
        Y=Y,
        opts=dict(legend=['Sine', 'Cosine'])
    )

'window_3bd20eb50e984a'

可以通过env参数指定Environments，如果名称包含了下划线_那么visdom会跟根据下划线分割并自动分组

In [5]:
envtest = Visdom(env='test_mesh')
assert envtest.check_connection()

Setting up a new session...


生成一个网格图

In [6]:
x = [0, 0, 1, 1, 0, 0, 1, 1]
y = [0, 1, 1, 0, 0, 1, 1, 0]
z = [0, 0, 0, 0, 1, 1, 1, 1]
X = np.c_[x, y, z]
i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2]
j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3]
k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6]
Y = np.c_[i, j, k]
envtest.mesh(X=X, Y=Y, opts=dict(opacity=0.5))

'window_3bd20ef5ab1254'

### 更新损失函数
在训练的时候我们每一批次都会打印一下训练的损失和测试的准确率，这样展示的图表是需要动态增加数据的，下面我们来模拟一下这种情况

In [7]:
x,y=0,0
env2 = Visdom()
pane1= env2.line(
    X=np.array([x]),
    Y=np.array([y]),
    opts=dict(title='dynamic data'))

Setting up a new session...


In [ ]:
for i in range(10):
    time.sleep(1) #每隔一秒钟打印一次数据
    x+=i
    y=(y+i)*1.5
    print(x,y)
    env2.line(
        X=np.array([x]),
        Y=np.array([y]),
        win=pane1,#win参数确认使用哪一个pane
        update='append') #我们做的动作是追加，除了追加意外还有其他方式，这里我们不做介绍了

45 464.9853515625
46 698.97802734375
48 1051.467041015625
51 1581.7005615234375
55 2378.5508422851562
60 3575.3262634277344
66 5371.989395141602
73 8068.484092712402
81 12114.726139068604
90 18185.589208602905


Connection to remote host was lost.
Connection to remote host was lost.
Connection to remote host was lost.
on_close() takes 1 positional argument but 3 were given
on_close() takes 1 positional argument but 3 were given
on_close() takes 1 positional argument but 3 were given
[Errno 111] Connection refused
on_close() takes 1 positional argument but 3 were given
[Errno 111] Connection refused
on_close() takes 1 positional argument but 3 were given
[Errno 111] Connection refused
on_close() takes 1 positional argument but 3 were given
[Errno 111] Connection refused
on_close() takes 1 positional argument but 3 were given
[Errno 111] Connection refused
on_close() takes 1 positional argument but 3 were given
[Errno 111] Connection refused
on_close() takes 1 positional argument but 3 were given
[Errno 111] Connection refused
on_close() takes 1 positional argument but 3 were given
[Errno 111] Connection refused
on_close() takes 1 positional argument but 3 were given
[Errno 111] Connection refus

在运行完上述程序时，切换到visdom，看看效果吧

visdom的基本用法介绍完毕，下一节介绍更加强大的 tensorboardx